https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/01_Adapter_Training.ipynb#scrollTo=hnct-N6dLvOd

In [1]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/scierc")
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3219 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/974 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [4]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

num_labels = len(set(train_data['labels']))

config = RobertaConfig.from_pretrained(
    "allenai/cs_roberta_base",
    num_labels=num_labels,
)
model = AutoAdapterModel.from_pretrained(
    "allenai/cs_roberta_base",
    config=config,
)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Basically, train_adapter() method performs two functions majorly:
*   It freezes all the weights of the pre-trained model such that only the adapter weights are updated during the training.
*   It also activates the adapter and prediction head to use both in every forward pass.

In [6]:
# Add a new adapter
model.add_adapter("scierc", config="par_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "scierc",
    num_labels=num_labels,
    id2label={
        0: "label_0",
        1: "label_1",
        2: "label_2",
        3: "label_3",
        4: "label_4",
        5: "label_5",
        6: "label_6",
    }
)

# Activate the adapter
model.train_adapter("scierc")

In [7]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [8]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,  # <- follow paper for smaller dataset
    num_train_epochs=5,  # <- paper used 100 epoch for TAPT instead
    per_device_train_batch_size=128,  # <- follow paper for smaller dataset
    per_device_eval_batch_size=128,  # <- follow paper for smaller dataset
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    adam_beta1=0.9,  # <- follow paper
    adam_beta2=0.98,  # <- follow paper
    adam_epsilon=1e-6,  # <- follow paper
    lr_scheduler_type='constant',  # <- follow paper for smaller dataset
    weight_decay=0.01,  # <- follow paper
    warmup_ratio=0.06,  # <- follow paper
    logging_steps=100,  # <- follow paper
    output_dir="./adapter_cs_roberta_base_par_bn_scierc_3",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [10]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,1.547400,1.638224,0.470330,0.067190,0.221210,0.142857,0.470330,0.091394,0.300899
2,1.114500,0.847729,0.709890,0.525039,0.629297,0.489682,0.709890,0.472778,0.633575
3,0.646600,0.496419,0.846154,0.792029,0.830898,0.719854,0.846154,0.740127,0.828574
4,0.423300,0.418027,0.874725,0.815987,0.893229,0.863601,0.874725,0.831596,0.879619
5,0.310800,0.406257,0.876923,0.833662,0.893637,0.845128,0.876923,0.832667,0.881797


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=505, training_loss=0.8023022030839826, metrics={'train_runtime': 691.0504, 'train_samples_per_second': 23.291, 'train_steps_per_second': 0.731, 'total_flos': 4614805157391360.0, 'train_loss': 0.8023022030839826, 'epoch': 5.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.40625691413879395,
 'eval_accuracy': 0.8769230769230769,
 'eval_precision_macro': 0.8336623914973034,
 'eval_precision_weighted': 0.8936369405636807,
 'eval_recall_macro': 0.8451280353166649,
 'eval_recall_weighted': 0.8769230769230769,
 'eval_f1_macro': 0.8326674989195492,
 'eval_f1_weighted': 0.8817968983262072,
 'eval_runtime': 8.4112,
 'eval_samples_per_second': 54.095,
 'eval_steps_per_second': 0.476,
 'epoch': 5.0}

In [13]:
trainer.predict(tokenized_test_data)

PredictionOutput(predictions=array([[ 0.9675882 , -1.8668915 ,  3.8547332 , ..., -1.9047438 ,
         0.52166665, -4.0225677 ],
       [ 1.0552306 , -2.7943935 ,  4.5344067 , ...,  2.1922615 ,
        -0.4018385 , -3.7488263 ],
       [ 1.7746809 , -3.1235251 ,  3.7373092 , ...,  1.9123603 ,
         0.11043289, -3.1699588 ],
       ...,
       [-2.0510256 ,  3.3855877 , -1.7019024 , ..., -2.909759  ,
         6.8129563 , -1.6335131 ],
       [-1.56409   , -0.42058647,  1.4689054 , ..., -2.4700966 ,
         1.3872524 , -3.658422  ],
       [-0.558916  ,  7.7176824 , -2.3231525 , ..., -2.8626952 ,
         0.90595794, -0.66328955]], dtype=float32), label_ids=array([2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 2, 5, 2, 0, 0, 0, 0, 2, 0, 0,
       0, 5, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 2,
       0, 0, 4, 0, 0, 4, 4, 1, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 5, 5, 5, 0,
       6, 5, 6, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 5, 5, 0, 0, 0, 5, 5, 0, 0,
       5, 0, 6, 6, 6, 0, 0, 0, 0, 0, 4, 